In [1]:
import linear_dag as ld
from linear_dag import LinearARG
from linear_dag.association.gwas import run_gwas
import time
import numpy as np
import polars as pl

from linear_dag.core.operators import get_merge_operator, get_inner_merge_operators

import matplotlib.pyplot as plt


In [3]:
linarg = ld.LinearARG.read('/Users/ambershen/Desktop/linarg/dx_analysis/test_pipeline/linear_args/csc/linear_arg_individual.h5')
# linarg = ld.LinearARG.read('/Users/ambershen/Desktop/linARG/dx_analysis/test_pipeline/linear_args/csc/linear_arg.h5')
N = int(linarg.shape[0]/2)

iids = pl.Series("whitelist", np.repeat(np.arange(N), 2)).cast(pl.Int64)
 
intercept = np.ones(N)
covar1 = np.random.normal(loc=0, scale=1, size=N)
phenotype = np.random.normal(loc=0, scale=1, size=N)

linarg.iids = iids

phenotypes = pl.DataFrame({
    "iid": np.arange(N),
    "intercept": intercept,
    "covar1": covar1,
    "phenotype1": phenotype,
    "phenotype2": phenotype,
})

phenotype = ['phenotype1', 'phenotype2']
covariates = ['intercept', 'covar1']

# phenotypes = phenotypes.slice(100)
phenotypes = phenotypes.slice(3200)

df = run_gwas(linarg, phenotypes.lazy(), phenotype, covariates, assume_hwe=False)

num_nonmissing: [4 4]
trait_series: shape: (8_118,)
Series: '' [enum]
[
	"phenotype1"
	"phenotype1"
	"phenotype1"
	"phenotype1"
	"phenotype1"
	…
	"phenotype2"
	"phenotype2"
	"phenotype2"
	"phenotype2"
	"phenotype2"
]
beta shape: (8118,)


In [7]:
df.collect()['variant_index']

variant_index
u32
0
1
2
3
4
…
4054
4055
4056


In [4]:
all_allele_counts = np.ones(linarg.shape[0]) @ linarg
all_carrier_counts = linarg.number_of_carriers()

allele_counts = np.ones(right_op.shape[0]) @ right_op @ linarg
rows_matched_per_col = np.ones(right_op.shape[0]) @ right_op
individuals_to_include = np.where(rows_matched_per_col[::2]==1)[0]
carrier_counts = linarg.number_of_carriers(individuals_to_include)

inds = np.where(allele_counts - carrier_counts < 0)

In [5]:
len(individuals_to_include)

2

In [6]:
np.where(all_allele_counts - all_carrier_counts < 0)

(array([], dtype=int64),)

In [7]:
np.where(all_allele_counts - allele_counts < 0), np.where(all_carrier_counts - carrier_counts < 0), inds

((array([], dtype=int64),),
 (array([], dtype=int64),),
 (array([], dtype=int64),))

In [8]:
np.where(all_allele_counts - all_carrier_counts < 0)

(array([], dtype=int64),)

In [9]:
allele_counts[inds][:10]

array([], dtype=float64)

In [10]:
carrier_counts[inds][:10]

array([], dtype=int64)

In [11]:
v = np.zeros(linarg.shape[0])
v[-1] = 1
v[-2] = 1
(v @ linarg)[inds][:10]

array([], dtype=float64)

In [12]:
v = np.zeros(linarg.shape[0])
v[-3] = 1
v[-4] = 1
(v @ linarg)[inds][:10]

array([], dtype=float64)

In [13]:
linarg.number_of_carriers(np.array([3201]))[inds][:10]

array([], dtype=int64)

In [14]:
linarg.number_of_carriers(np.array([3200]))[inds][:10]

array([], dtype=int64)

In [15]:
v = np.zeros(linarg.shape[0])
v[-1] = 1
v[-2] = 1

print(linarg.number_of_carriers(np.array([3201]))[:40])
print((v @ linarg)[:40].astype(int))

[0 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 0 1 1 1 1 0 1 1 0 1 1 1 1 1 1 1 1 1 1 1
 1 1 1]
[0 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 0 1 1 2 2 0 1 1 0 1 1 1 2 1 1 1 1 2 2 2
 1 1 1]


In [16]:
col_index = 32325
linarg.A[:, col_index].nonzero()[0], linarg.A[col_index, :].nonzero()[1]

(array([], dtype=int32),
 array([ 1299,  1317,  1363,  1492,  1643,  1851,  1980,  2006,  2399,
         2600,  2684,  3491,  3548,  3564,  4088,  4092,  4103,  4114,
         4210,  4231,  4259,  4510,  4597,  4646,  4879,  4917,  5261,
         5388,  5467,  5564,  5764,  5835,  6234,  6264,  7167,  7189,
         7242,  7265,  7417,  7464,  7565,  9766,  9793, 10912, 11425,
        11428, 11432, 11441, 11442, 11489, 11503, 11505, 11507, 11528,
        11533, 11563, 11565, 11580, 11590, 11592, 11594, 13879, 14000,
        16920, 18469, 21169, 22720, 22721], dtype=int32))

In [16]:
linarg.sample_indices

array([29123, 29122, 29121, ..., 22722, 22721, 22720], dtype=int32)

In [11]:
linarg.individual_indices

array([32325, 32324, 32323, ..., 29126, 29125, 29124], dtype=int32)

In [13]:
linarg.A[:, col_index].nonzero()

(array([], dtype=int32), array([], dtype=int32))

In [7]:
linarg.A

<Compressed Sparse Column sparse matrix of dtype 'int32'
	with 346460 stored elements and shape (32326, 32326)>

In [5]:
left, right = get_inner_merge_operators(pl.Series(np.array([0, 1, 3, 4])), pl.Series(np.repeat(np.arange(5), 2)))
left @ np.eye(left.shape[1])

array([[1., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 1.]])

In [6]:
right @ np.eye(right.shape[1])

array([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]])

In [10]:
np.where((np.ones(right.shape[0]) @ right)[::2]==1)[0]

array([0, 1, 3, 4])

In [22]:
merge1 = get_merge_operator(pl.Series(np.arange(5)), pl.Series(np.repeat(np.arange(5), 2)))
merge1 @ np.eye(10)

array([[1., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 1.]])

In [23]:
merge2 = get_merge_operator(pl.Series(np.array([0, 1, 3, 4])), pl.Series(np.repeat(np.arange(5), 2)))
merge2 @ np.eye(10)

array([[1., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 1.]])

In [31]:
phenotypes = np.array([2, 4, 6, 8])

merge2.T @ phenotypes

array([2, 2, 4, 4, 0, 0, 6, 6, 8, 8])

In [26]:
row_ids = pl.Series(np.array([0, 1, 3, 4]))
col_ids = pl.Series(np.repeat(np.arange(5), 2))


row_df = pl.LazyFrame({'id': row_ids})\
    .with_row_index('row_idx')
col_df = pl.LazyFrame({'id': col_ids})\
    .with_row_index('col_idx')
merged_df = row_df.join(col_df, on='id', how='inner').collect()

In [27]:
merged_df

row_idx,id,col_idx
u32,i64,u32
0,0,0
0,0,1
1,1,2
1,1,3
2,3,6
2,3,7
3,4,8
3,4,9
